<a href="https://colab.research.google.com/github/alvaroalon2/NER4COVID/blob/main/NER4COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.0MB 22.4MB/s 
     |████████████████████████████████| 890kB 44.2MB/s 
     |████████████████████████████████| 3.2MB 51.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=cd9aacc8be0144db0060b38f97ad78247e3a2a411923571c600343c92e595eea
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [6]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=312af56fdc298559b9e728564d0881450bf466391265d00bd2e1ce179027fa63
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [7]:
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel, BertTokenizerFast
import torch
import tensorflow as tf
import os

# Preparing GPU

In [3]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [4]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
!nvidia-smi

Thu Apr  1 14:08:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |    222MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Corpus preparation

## GENIA corpus

In [11]:
import wget

url = "http://www.nactem.ac.uk/GENIA/current/GENIA-corpus/Term/GENIAcorpus3.02.tgz"
if not os.path.exists('./GENIAcorpus3.02.tgz'):
  wget.download(url,'./GENIAcorpus3.02.tgz')

In [13]:
if not os.path.exists('./GENIA_term_3.02'):
  !tar -xvzf GENIAcorpus3.02.tgz

GENIA_term_3.02/
GENIA_term_3.02/gpml.css
GENIA_term_3.02/LICENSE
GENIA_term_3.02/GENIAcorpus3.02.xml
GENIA_term_3.02/gpml.dtd
GENIA_term_3.02/gpml.css.legend.html
GENIA_term_3.02/gpml.readme.html


# Model preparation

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

In [ ]:
print('tokenizer type:', type(tokenizer))
print('model type:', type(model))

tokenizer type: <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
model type: <class 'transformers.models.bert.modeling_bert.BertModel'>


In [ ]:
text1 = "Hydrogels are hydrophilic polymer networks which may absorb from " \
       "10–20% (an arbitrary lower limit) up to thousands of times their " \
       "dry weight in water."

In [ ]:
text2 = "SARS-CoV-2 variants of concern (VOC) have arisen independently at multiple locations [1, 2]" \
        "and may reduce the efficacy of current vaccines targeting the spike glycoprotein [3]. Here, using a live virus neutralization assay (LVNA)," \
        " we compared neutralization of a non-VOC variant versus the 501Y.V2 variant using plasma collected from adults hospitalized with COVID-19" \
        " from two South African infection waves, with the second wave dominated by 501Y.V2 infections."

In [ ]:
tokens = tokenizer.tokenize(text2)

In [ ]:
for token in tokens:
  print(token + '\n')

sars

-

cov

-

2

variants

of

concern

(

voc

)

have

arisen

independently

at

multiple

locations

[

1

,

2

]

and

may

reduce

the

efficacy

of

current

vaccines

targeting

the

spike

glycoprotein

[

3

]

.

here

,

using

a

live

virus

neutralization

assay

(

lv

##na

)

,

we

compared

neutralization

of

a

non

-

voc

variant

versus

the

501

##y

.

v2

variant

using

plasma

collected

from

adults

hospitalized

with

cov

##id

-

19

from

two

south

african

infection

waves

,

with

the

second

wave

dominated

by

501

##y

.

v2

infections

.

